In [ ]:
#Part 1: Set Up Model Group

In [1]:
import boto3

client = boto3.client('sagemaker')

group_name = "facial-expression-analysis-aai540"
description = "MobileNetV2 model to classify facial expressions for neuromarketing using FERPlus and RAF-DB."

client.create_model_package_group(
    ModelPackageGroupName=group_name,
    ModelPackageGroupDescription=description
)

ClientError: An error occurred (ValidationException) when calling the CreateModelPackageGroup operation: Model Package Group already exists: arn:aws:sagemaker:us-east-1:877305355343:model-package-group/facial-expression-analysis-aai540

In [2]:
response = client.describe_model_package_group(ModelPackageGroupName=group_name)
print(response)

{'ModelPackageGroupName': 'facial-expression-analysis-aai540', 'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:877305355343:model-package-group/facial-expression-analysis-aai540', 'ModelPackageGroupDescription': 'MobileNetV2 model to classify facial expressions for neuromarketing using FERPlus and RAF-DB.', 'CreationTime': datetime.datetime(2025, 6, 17, 1, 44, 36, 82000, tzinfo=tzlocal()), 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:877305355343:user-profile/d-pouurhqhlhjk/default-20250607T190108', 'UserProfileName': 'default-20250607T190108', 'DomainId': 'd-pouurhqhlhjk', 'IamIdentity': {'Arn': 'arn:aws:sts::877305355343:assumed-role/AmazonSageMaker-ExecutionRole-20250607T190109/SageMaker', 'PrincipalId': 'AROA4YQ3ZLRHVDVH366J5:SageMaker'}}, 'ModelPackageGroupStatus': 'Completed', 'ResponseMetadata': {'RequestId': '43130e58-db84-4a31-b0eb-4662fb2cf6b0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '43130e58-db84-4a31-b0eb-4662fb2cf6b0', 'content-typ

In [ ]:
#Part 2: Set up Model Package

In [3]:
model_package_response = client.create_model_package(
    ModelPackageGroupName=group_name,
    ModelPackageDescription="Facial expression analysis model using MobileNetV2.",
    InferenceSpecification={
        'Containers': [{
            'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.9.1-cpu-py38-ubuntu20.04',
            'ModelDataUrl': 's3://sagemaker-us-east-1-877305355343/mobV2_model.tar.gz',
            'Environment': {}
        }],
        'SupportedContentTypes': ['application/x-image'],
        'SupportedResponseMIMETypes': ['application/json']
    },
    CertifyForMarketplace=False
)

In [4]:
model_package_name = model_package_response['ModelPackageArn']

response = client.describe_model_package(ModelPackageName=model_package_name)
print(response)

{'ModelPackageGroupName': 'facial-expression-analysis-aai540', 'ModelPackageVersion': 1, 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:877305355343:model-package/facial-expression-analysis-aai540/1', 'ModelPackageDescription': 'Facial expression analysis model using MobileNetV2.', 'CreationTime': datetime.datetime(2025, 6, 17, 1, 49, 39, 959000, tzinfo=tzlocal()), 'InferenceSpecification': {'Containers': [{'Image': '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.9.1-cpu-py38-ubuntu20.04', 'ImageDigest': 'sha256:327d9112f93764b3d94edf7e2629d9a4d4dc410ccce0e7d8e3a8f4b4d9cdc91c', 'ModelDataUrl': 's3://sagemaker-us-east-1-877305355343/mobV2_model.tar.gz', 'Environment': {}, 'ModelDataETag': 'e2f76cad2c8e785ec046b3d57528f9a6'}], 'SupportedContentTypes': ['application/x-image'], 'SupportedResponseMIMETypes': ['application/json']}, 'ModelPackageStatus': 'Completed', 'ModelPackageStatusDetails': {'ValidationStatuses': [], 'ImageScanStatuses': []}, 'CertifyForMarketplace': F

In [ ]:
#Part 3: Write a Model Card

In [11]:
import json

model_card_content = {
    "model_overview": {
        "model_description": "MobileNetV2 model trained to classify facial expressions into 7 emotions using FERPlus and RAF-DB datasets.",
        "model_owner": "Arifa Kokab",
        "problem_type": "Image Classification",
        "algorithm_type": "Convolutional Neural Network"
    },
    "intended_uses": {
        "intended_uses": "Neuromarketing teams will use this model to analyze emotional responses from facial expressions across video content."
    },
    "training_details": {
        "training_observations": "The model was trained using stratified and balanced data splits from FERPlus and RAF-DB. Image augmentation and class weights were used."
    },
    "evaluation_details": [
        {
            "name": "Validation Accuracy",
            "evaluation_job_arn": "arn:aws:sagemaker:us-east-1:123456789012:transform-job/facial-expression-eval-job",
            "evaluation_observation": "Validation accuracy achieved 89.6% on combined FERPlus and RAF-DB validation set."
        },
        {
            "name": "Macro F1 Score",
            "evaluation_job_arn": "arn:aws:sagemaker:us-east-1:123456789012:transform-job/facial-expression-eval-job",
            "evaluation_observation": "Macro average F1 score was 0.831 as calculated using sklearn metrics."
        }
    ]
}

In [12]:
client.create_model_card(
    ModelCardName='facial-expression-analysis-card',
    Content=json.dumps(model_card_content),
    ModelCardStatus='Draft'
)

{'ModelCardArn': 'arn:aws:sagemaker:us-east-1:877305355343:model-card/facial-expression-analysis-card',
 'ResponseMetadata': {'RequestId': '613d3e6e-59cd-4fe7-bcdc-a3644e4a13c4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '613d3e6e-59cd-4fe7-bcdc-a3644e4a13c4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '102',
   'date': 'Tue, 17 Jun 2025 01:55:30 GMT'},
  'RetryAttempts': 0}}

In [13]:
response = client.describe_model_card(ModelCardName='facial-expression-analysis-card')
print(response)

{'ModelCardArn': 'arn:aws:sagemaker:us-east-1:877305355343:model-card/facial-expression-analysis-card', 'ModelCardName': 'facial-expression-analysis-card', 'ModelCardVersion': 1, 'Content': '{"model_overview": {"model_description": "MobileNetV2 model trained to classify facial expressions into 7 emotions using FERPlus and RAF-DB datasets.", "model_owner": "Arifa Kokab", "problem_type": "Image Classification", "algorithm_type": "Convolutional Neural Network"}, "intended_uses": {"intended_uses": "Neuromarketing teams will use this model to analyze emotional responses from facial expressions across video content."}, "training_details": {"training_observations": "The model was trained using stratified and balanced data splits from FERPlus and RAF-DB. Image augmentation and class weights were used."}, "evaluation_details": [{"name": "Validation Accuracy", "evaluation_job_arn": "arn:aws:sagemaker:us-east-1:123456789012:transform-job/facial-expression-eval-job", "evaluation_observation": "Val